# Critical Edge Case Testing - Stock Adjustment Barcode
## Focus: Backdated Refresh & Lot Difference Handling

This notebook tests **critical scenarios that could break** based on code analysis:

### Critical Issues Identified:
1. **Backdated refresh with stock movements between backdate and scan**
2. **Lot difference calculation with mixed positive/negative quants**
3. **FIFO lot selection edge cases (empty lots, single lot, all negative)**
4. **Quant gathering failures and fallback scenarios**
5. **Difference mismatch between line and lot totals**
6. **Index out of bounds in FIFO deduction loop**
7. **Precision rounding errors in difference calculations**
8. **State inconsistencies during refresh**

## Setup

In [27]:
import sys
import os
from datetime import datetime, timedelta
from decimal import Decimal
import pandas as pd
import json

sys.path.append('/Users/jamshid/PycharmProjects/Siafa/src')

import odoo
from odoo import api, fields, models, SUPERUSER_ID
from odoo.exceptions import UserError, ValidationError
from odoo.tools import float_compare, float_round, float_is_zero

# Configure Odoo
odoo.tools.config.parse_config(['--config=/Users/jamshid/PycharmProjects/Siafa/src/odoo.conf'])
db_name = 'oct_8'
uid = SUPERUSER_ID
context = {"company_id": 2}

registry = odoo.registry(db_name)

# Test Configuration
CRITICAL_TEST_CONFIG = {
    'company_id': 2 ,
    'location_id': 1807,
    'test_product_id': 19536,  # Lot-tracked product
    'test_product_barcode': '6281106251280',
}

print("Critical Test Environment Loaded")

Critical Test Environment Loaded


## CRITICAL TEST 1: Backdated Refresh with Intermediate Stock Movements
### Risk: Stock calculated at backdate may not match current scanned reality

**Scenario:**
- T-10 days: Stock = 100 units
- T-7 days: Backdate point
- T-5 days: +50 units added
- T-3 days: -30 units removed
- T-0 days: User scans expecting 120 units
- **Issue:** Refresh to T-7 shows 100, but user scanned based on 120

In [28]:
with registry.cursor() as cr:
    env = api.Environment(cr, uid, context)
    
    print("=" * 70)
    print("CRITICAL TEST 1: Backdated Refresh with Intermediate Movements")
    print("=" * 70)
    
    # Setup: Create product and lots
    product = env['product.product'].browse(CRITICAL_TEST_CONFIG['test_product_id'])
    location = env['stock.location'].browse(CRITICAL_TEST_CONFIG['location_id'])
    
    # Create lot
    lot = env['stock.lot'].create({
        'name': 'CRITICAL-LOT-001',
        'product_id': product.id,
        'company_id': CRITICAL_TEST_CONFIG['company_id'],
    })
    
    # T-10: Initial stock
    date_t10 = datetime.now() - timedelta(days=10)
    quant = env['stock.quant'].create({
        'product_id': product.id,
        'location_id': location.id,
        'lot_id': lot.id,
        'quantity': 100,
        'in_date': date_t10,
        'company_id': CRITICAL_TEST_CONFIG['company_id'],
    })
    print(f"T-10 days: Initial stock = 100 units")
    
    # T-5: Add 50 units using stock picking
    date_t5 = datetime.now() - timedelta(days=5)
    inventory_loc = product.property_stock_inventory
    
    # Get picking type for internal transfers
    picking_type = env['stock.picking.type'].search([
        ('code', '=', 'internal'),
        ('company_id', '=', CRITICAL_TEST_CONFIG['company_id'])
    ], limit=1)
    
    # Create incoming picking
    picking_in = env['stock.picking'].create({
        'picking_type_id': picking_type.id,

        'location_dest_id': location.id,
        'scheduled_date': date_t5,
        'company_id': CRITICAL_TEST_CONFIG['company_id'],
    })
    
    # Create move in picking
    move_in = env['stock.move'].create({
        'name': 'Test Move In',
        'product_id': product.id,
        'product_uom_qty': 50,
        'product_uom': product.uom_id.id,
        'picking_id': picking_in.id,
        
        'location_id': picking_in.location_id.id,
        'location_dest_id': location.id,
        'date': date_t5,
    })
    
    # Process picking
    picking_in.action_confirm()
    picking_in.action_assign()
    picking_in.move_line_ids.write({'lot_id': lot.id, 'qty_done': 50})
    picking_in.button_validate()
    print(f"T-5 days: Added 50 units via picking, total should be 150")
    
    # T-3: Remove 30 units using stock picking
    date_t3 = datetime.now() - timedelta(days=3)
    
    # Create outgoing picking
    picking_out = env['stock.picking'].create({
        'picking_type_id': picking_type.id,
        'location_id': location.id,
        'location_dest_id': inventory_loc.id,
        'scheduled_date': date_t3,
        'company_id': CRITICAL_TEST_CONFIG['company_id'],
    })
    
    # Create move in picking
    move_out = env['stock.move'].create({
        'name': 'Test Move Out',
        'product_id': product.id,
        'product_uom_qty': 30,
        'product_uom': product.uom_id.id,
        'picking_id': picking_out.id,
        'company_id': CRITICAL_TEST_CONFIG['company_id'],
        'location_id': location.id,
        'location_dest_id': inventory_loc.id,
        'date': date_t3,
    })
    
    # Process picking
    picking_out.action_confirm()
    picking_out.action_assign()
    picking_out.move_line_ids.write({'lot_id': lot.id, 'qty_done': 30})
    picking_out.button_validate()
    print(f"T-3 days: Removed 30 units via picking, current stock = 120")
    print(f"T-3 days: Removed 30 units, current stock = 120")
    
    # Current stock check
    current_quants = env['stock.quant'].search([
        ('product_id', '=', product.id),
        ('location_id', '=', location.id),
        ('lot_id', '=', lot.id)
    ])
    current_stock = sum(current_quants.mapped('quantity'))
    print(f"\nCurrent stock (T-0): {current_stock} units")
    
    # Create adjustment and scan based on current stock
    adjustment = env['stock.adjustment.barcode'].create({
        'location_id': location.id,
        'company_id': CRITICAL_TEST_CONFIG['company_id'],
    })
    
    # User scans 120 (expecting current stock)
    for i in range(120):
        adjustment.on_barcode_scanned(CRITICAL_TEST_CONFIG['test_product_barcode'])
    
    print(f"\nUser scanned: 120 times (based on current stock)")
    
    # Confirm - uses current stock
    adjustment.action_confirm()
    
    line = adjustment.inv_adjustment_line_ids[0]
    print(f"\nAfter Confirm (current stock):")
    print(f"  On Hand: {line.on_hand_qty}")
    print(f"  Scanned: {line.total_scanned_qty}")
    print(f"  Difference: {line.difference_qty}")
    
    # NOW REFRESH TO T-7 (backdated)
    backdate = datetime.now() - timedelta(days=7)
    print(f"\n{'='*70}")
    print(f"APPLYING BACKDATED REFRESH TO T-7 days...")
    print(f"{'='*70}")
    
    wizard = env['stock.refresh.stock.wizard'].with_context(
        active_model='stock.adjustment.barcode',
        active_id=adjustment.id
    ).create({
        'fetch_type': 'backdated',
        'backdate': backdate
    })
    
    wizard.action_apply()
    
    # Reload line
    line = adjustment.inv_adjustment_line_ids[0]
    
    print(f"\nAfter Backdated Refresh (T-7):")
    print(f"  On Hand (backdated): {line.on_hand_qty}")
    print(f"  Scanned: {line.total_scanned_qty}")
    print(f"  Difference: {line.difference_qty}")
    print(f"  Has Error: {line.is_editable}")
    
    # Check lot lines
    print(f"\nLot Lines After Refresh:")
    for lot_line in line.adjustment_line_lot_ids:
        print(f"  Lot: {lot_line.lot_id.name}")
        print(f"    Current: {lot_line.current_qty}")
        print(f"    New: {lot_line.new_qty}")
        print(f"    Difference: {lot_line.difference_qty}")
    
    # CRITICAL VALIDATION
    print(f"\n{'='*70}")
    print("CRITICAL VALIDATION:")
    print(f"{'='*70}")
    
    expected_backdate_stock = 100  # At T-7, only initial 100 existed
    actual_backdate_stock = line.on_hand_qty
    
    if actual_backdate_stock == expected_backdate_stock:
        print(f"✅ PASS: Backdated stock correctly shows {expected_backdate_stock}")
    else:
        print(f"❌ FAIL: Expected {expected_backdate_stock}, got {actual_backdate_stock}")
    
    # Check if difference is reasonable
    expected_diff = 120 - 100  # Scanned 120, backdated stock is 100
    actual_diff = line.difference_qty
    
    if actual_diff == expected_diff:
        print(f"✅ PASS: Difference correctly calculated as {expected_diff}")
    else:
        print(f"❌ FAIL: Expected difference {expected_diff}, got {actual_diff}")
    
    # Check lot line totals match line difference
    lot_total_diff = sum(line.adjustment_line_lot_ids.mapped('difference_qty'))
    precision = env['decimal.precision'].precision_get('Product Unit of Measure')
    
    if float_compare(lot_total_diff, line.difference_qty, precision_digits=precision) == 0:
        print(f"✅ PASS: Lot differences sum matches line difference")
    else:
        print(f"❌ FAIL: Lot total {lot_total_diff} != Line difference {line.difference_qty}")
        print(f"   CRITICAL: This will cause validation error on posting!")
    
    cr.rollback()  # Don't commit test data

2025-10-15 12:22:34,666 89074 DEBUG oct_8 odoo.tools.translate: no translation language detected, skipping translation for "'%s created'"  
2025-10-15 12:22:34,692 89074 DEBUG oct_8 odoo.tools.translate: no translation language detected, skipping translation for "'%s created'"  
2025-10-15 12:22:34,730 89074 DEBUG oct_8 odoo.tools.translate: no translation language detected, skipping translation for "'Incompatible companies on records:'"  
2025-10-15 12:22:34,731 89074 DEBUG oct_8 odoo.tools.translate: no translation language detected, skipping translation for "'- %(record)r belongs to company %(company)r and %(field)r (%(fname)s: %(values)s) belongs to another company.'"  
2025-10-15 12:22:34,733 89074 DEBUG oct_8 odoo.tools.translate: no translation language detected, skipping translation for "'- %(record)r belongs to company %(company)r and %(field)r (%(fname)s: %(values)s) belongs to another company.'"  
2025-10-15 12:22:34,735 89074 DEBUG oct_8 odoo.tools.translate: no translation

CRITICAL TEST 1: Backdated Refresh with Intermediate Movements
T-10 days: Initial stock = 100 units


UserError: Incompatible companies on records:
- '15000691: Jeddah Factory>Makkah Retail Arjoon Basta 1' belongs to company 'Siafa Group' and 'Product' (product_id: 'Safawi Almond Single Wrapped 5 kg') belongs to another company.
- '15000691: Jeddah Factory>Makkah Retail Arjoon Basta 1' belongs to company 'Siafa Group' and 'Source Location' (location_id: 'JDFCW/Jeddah Factory') belongs to another company.
- '15000691: Jeddah Factory>Makkah Retail Arjoon Basta 1' belongs to company 'Siafa Group' and 'Destination Location' (location_dest_id: 'MKABS/Makkah Retail Arjoon Basta 1') belongs to another company.
- '15000691: Jeddah Factory>Makkah Retail Arjoon Basta 1' belongs to company 'Siafa Group' and 'Transfer' (picking_id: 'JDFCW/SCRAPD/00088') belongs to another company.
- '15000691: Jeddah Factory>Makkah Retail Arjoon Basta 1' belongs to company 'Siafa Group' and 'Operation Type' (picking_type_id: 'Jeddah Factory Warehouse: Finished Goods -> Scrapped (D)') belongs to another company.

## CRITICAL TEST 2: FIFO Deduction with Index Out of Bounds
### Risk: Line 266-274 in set_lot_on_adjustment_line_ids - while loop may exceed quant list

**Code at Risk:**
```python
quant_index = 0
taken_quantity = abs(difference)
while (taken_quantity > 0):
    quant = positive_quants[quant_index]  # Could exceed list!
    ...
    quant_index += 1
```

In [ ]:
with registry.cursor() as cr:
    env = api.Environment(cr, uid, context)
    
    print("=" * 70)
    print("CRITICAL TEST 2: FIFO Index Out of Bounds")
    print("=" * 70)
    
    product = env['product.product'].browse(CRITICAL_TEST_CONFIG['test_product_id'])
    location = env['stock.location'].browse(CRITICAL_TEST_CONFIG['location_id'])
    
    # Create 2 lots with small quantities
    lot1 = env['stock.lot'].create({'name': 'FIFO-LOT-1', 'product_id': product.id, 'company_id': CRITICAL_TEST_CONFIG['company_id']})
    lot2 = env['stock.lot'].create({'name': 'FIFO-LOT-2', 'product_id': product.id, 'company_id': CRITICAL_TEST_CONFIG['company_id']})
    
    # Create quants with SMALL total
    date1 = datetime.now() - timedelta(days=5)
    date2 = datetime.now() - timedelta(days=3)
    
    env['stock.quant'].create({
        'product_id': product.id,
        'location_id': location.id,
        'lot_id': lot1.id,
        'quantity': 5,  # Small quantity
        'in_date': date1,
        'company_id': CRITICAL_TEST_CONFIG['company_id'],
    })
    
    env['stock.quant'].create({
        'product_id': product.id,
        'location_id': location.id,
        'lot_id': lot2.id,
        'quantity': 3,  # Even smaller
        'in_date': date2,
        'company_id': CRITICAL_TEST_CONFIG['company_id'],
    })
    
    total_stock = 8
    print(f"Setup: LOT1=5, LOT2=3, Total={total_stock}")
    
    # Create adjustment - scan 0 to force deduction of ALL stock
    adjustment = env['stock.adjustment.barcode'].create({
        'location_id': location.id,
        'company_id': CRITICAL_TEST_CONFIG['company_id'],
    })
    
    # Scan 0 times (or create line with 0) - will try to deduct 8 units
    line_info = env['stock.adjustment.barcode.line.info'].create({
        'inv_adjustment_id': adjustment.id,
        'product_id': product.id,
        'scanned_qty': 0,
        'scanned_user_id': env.user.id,
    })
    
    print(f"\nScanned: 0 (will deduct all {total_stock} units)")
    
    # Confirm - this triggers set_lot_on_adjustment_line_ids
    print(f"\nConfirming adjustment (triggers FIFO deduction)...")
    
    try:
        adjustment.action_confirm()
        
        line = adjustment.inv_adjustment_line_ids[0]
        print(f"\n✅ PASS: No index error occurred")
        print(f"  Difference: {line.difference_qty}")
        print(f"  Lot lines created: {len(line.adjustment_line_lot_ids)}")
        
        # Verify all lots processed
        print(f"\nLot Lines:")
        lot_total = 0
        for lot_line in line.adjustment_line_lot_ids:
            print(f"  {lot_line.lot_id.name}: current={lot_line.current_qty}, new={lot_line.new_qty}, diff={lot_line.difference_qty}")
            lot_total += lot_line.difference_qty
        
        # Critical check: lot totals match
        precision = env['decimal.precision'].precision_get('Product Unit of Measure')
        if float_compare(lot_total, line.difference_qty, precision_digits=precision) == 0:
            print(f"\n✅ PASS: Lot totals match line difference")
        else:
            print(f"\n❌ FAIL: Lot total {lot_total} != Line diff {line.difference_qty}")
        
    except IndexError as e:
        print(f"\n❌ CRITICAL FAIL: IndexError occurred!")
        print(f"   Error: {str(e)}")
        print(f"   This means quant_index exceeded positive_quants list length")
    except Exception as e:
        print(f"\n⚠️ Other error: {str(e)}")
    
    cr.rollback()

## CRITICAL TEST 3: Mixed Negative and Positive Quants with Exact Deduction
### Risk: Complex logic in lines 219-282 handling negative quants + positive quants

**Scenario:**
- LOT1: -5 (negative stock)
- LOT2: +10
- LOT3: -3 (negative)
- LOT4: +8
- Total: 10 units
- Scan: 5 units
- Expected: All negative become 0, then deduct 5 from positive

In [ ]:
with registry.cursor() as cr:
    env = api.Environment(cr, uid, context)
    
    print("=" * 70)
    print("CRITICAL TEST 3: Mixed Negative/Positive Quants")
    print("=" * 70)
    
    product = env['product.product'].browse(CRITICAL_TEST_CONFIG['test_product_id'])
    location = env['stock.location'].browse(CRITICAL_TEST_CONFIG['location_id'])
    
    # Create lots
    lots = []
    for i in range(1, 5):
        lot = env['stock.lot'].create({
            'name': f'MIX-LOT-{i}',
            'product_id': product.id,
            'company_id': CRITICAL_TEST_CONFIG['company_id'],
        })
        lots.append(lot)
    
    # Create mixed quants
    base_date = datetime.now() - timedelta(days=10)
    quant_configs = [
        (lots[0], -5, 0),   # Negative
        (lots[1], 10, 1),   # Positive
        (lots[2], -3, 2),   # Negative
        (lots[3], 8, 3),    # Positive
    ]
    
    print("Setup:")
    total_stock = 0
    for lot, qty, day_offset in quant_configs:
        env['stock.quant'].create({
            'product_id': product.id,
            'location_id': location.id,
            'lot_id': lot.id,
            'quantity': qty,
            'in_date': base_date + timedelta(days=day_offset),
            'company_id': CRITICAL_TEST_CONFIG['company_id'],
        })
        total_stock += qty
        print(f"  {lot.name}: {qty:+3d}")
    
    print(f"\nTotal Stock: {total_stock}")
    
    # Create adjustment and scan 5 units
    adjustment = env['stock.adjustment.barcode'].create({
        'location_id': location.id,
        'company_id': CRITICAL_TEST_CONFIG['company_id'],
    })
    
    scan_qty = 5
    for i in range(scan_qty):
        adjustment.on_barcode_scanned(CRITICAL_TEST_CONFIG['test_product_barcode'])
    
    print(f"\nScanned: {scan_qty} units")
    print(f"Expected difference: {scan_qty - total_stock} = {scan_qty} - {total_stock}")
    
    # Confirm
    adjustment.action_confirm()
    
    line = adjustment.inv_adjustment_line_ids[0]
    
    print(f"\nAfter Confirm:")
    print(f"  On Hand: {line.on_hand_qty}")
    print(f"  Scanned: {line.total_scanned_qty}")
    print(f"  Difference: {line.difference_qty}")
    print(f"  Has Error: {line.is_editable}")
    
    # Check lot lines
    print(f"\nLot Lines:")
    lot_diff_total = 0
    negative_lots_zeroed = True
    
    for lot_line in line.adjustment_line_lot_ids.sorted('lot_id'):
        print(f"  {lot_line.lot_id.name}:")
        print(f"    Current: {lot_line.current_qty:+6.2f}")
        print(f"    New:     {lot_line.new_qty:+6.2f}")
        print(f"    Diff:    {lot_line.difference_qty:+6.2f}")
        
        lot_diff_total += lot_line.difference_qty
        
        # Check if negative quants are zeroed
        if lot_line.current_qty < 0 and lot_line.new_qty != 0:
            negative_lots_zeroed = False
    
    print(f"\nLot differences total: {lot_diff_total}")
    
    # CRITICAL VALIDATIONS
    print(f"\n{'='*70}")
    print("CRITICAL VALIDATIONS:")
    print(f"{'='*70}")
    
    precision = env['decimal.precision'].precision_get('Product Unit of Measure')
    
    # 1. All negative quants should become 0
    if negative_lots_zeroed:
        print(f"✅ PASS: All negative quants set to 0")
    else:
        print(f"❌ FAIL: Some negative quants not zeroed")
    
    # 2. Lot totals should match line difference
    if float_compare(lot_diff_total, line.difference_qty, precision_digits=precision) == 0:
        print(f"✅ PASS: Lot totals match line difference")
    else:
        print(f"❌ CRITICAL FAIL: Lot total {lot_diff_total} != Line {line.difference_qty}")
        print(f"   This will cause _check_line_difference_match() to fail!")
    
    # 3. Final stock should equal scanned
    expected_final = scan_qty
    actual_final = sum(line.adjustment_line_lot_ids.mapped('new_qty'))
    
    if float_compare(actual_final, expected_final, precision_digits=precision) == 0:
        print(f"✅ PASS: Final stock matches scanned quantity")
    else:
        print(f"❌ FAIL: Final stock {actual_final} != Scanned {expected_final}")
    
    cr.rollback()

## CRITICAL TEST 4: Precision Rounding Errors
### Risk: float_round in lines 168, 272 - accumulation of rounding errors

In [ ]:
with registry.cursor() as cr:
    env = api.Environment(cr, uid, context)
    
    print("=" * 70)
    print("CRITICAL TEST 4: Precision Rounding Errors")
    print("=" * 70)
    
    product = env['product.product'].browse(CRITICAL_TEST_CONFIG['test_product_id'])
    location = env['stock.location'].browse(CRITICAL_TEST_CONFIG['location_id'])
    
    # Create lots with decimal quantities that cause rounding issues
    lots = []
    for i in range(1, 6):
        lot = env['stock.lot'].create({
            'name': f'PRECISION-LOT-{i}',
            'product_id': product.id,
            'company_id': CRITICAL_TEST_CONFIG['company_id'],
        })
        lots.append(lot)
    
    # Create quants with problematic decimal values
    base_date = datetime.now() - timedelta(days=5)
    decimal_quantities = [3.333333, 2.666667, 1.999999, 4.111111, 2.888889]
    
    print("Setup with decimal quantities:")
    total = 0
    for i, (lot, qty) in enumerate(zip(lots, decimal_quantities)):
        env['stock.quant'].create({
            'product_id': product.id,
            'location_id': location.id,
            'lot_id': lot.id,
            'quantity': qty,
            'in_date': base_date + timedelta(days=i),
            'company_id': CRITICAL_TEST_CONFIG['company_id'],
        })
        total += qty
        print(f"  {lot.name}: {qty:.6f}")
    
    print(f"\nTotal (raw sum): {total:.6f}")
    
    precision = env['decimal.precision'].precision_get('Product Unit of Measure')
    total_rounded = float_round(total, precision_digits=precision)
    print(f"Total (rounded to {precision} decimals): {total_rounded}")
    
    # Scan exactly the rounded total
    adjustment = env['stock.adjustment.barcode'].create({
        'location_id': location.id,
        'company_id': CRITICAL_TEST_CONFIG['company_id'],
    })
    
    scan_qty = int(total_rounded)  # Scan integer version
    for i in range(scan_qty):
        adjustment.on_barcode_scanned(CRITICAL_TEST_CONFIG['test_product_barcode'])
    
    print(f"\nScanned: {scan_qty} (integer of rounded total)")
    
    # Confirm
    adjustment.action_confirm()
    
    line = adjustment.inv_adjustment_line_ids[0]
    
    print(f"\nAfter Confirm:")
    print(f"  On Hand: {line.on_hand_qty:.6f}")
    print(f"  Scanned: {line.total_scanned_qty:.6f}")
    print(f"  Difference: {line.difference_qty:.6f}")
    
    # Check lot lines
    print(f"\nLot Lines:")
    lot_diff_sum = 0
    for lot_line in line.adjustment_line_lot_ids:
        print(f"  {lot_line.lot_id.name}: diff={lot_line.difference_qty:.6f}")
        lot_diff_sum += lot_line.difference_qty
    
    print(f"\nLot differences sum: {lot_diff_sum:.6f}")
    print(f"Line difference:     {line.difference_qty:.6f}")
    
    # CRITICAL: Check if they match with precision
    print(f"\n{'='*70}")
    print("ROUNDING ERROR CHECK:")
    print(f"{'='*70}")
    
    if float_compare(lot_diff_sum, line.difference_qty, precision_digits=precision) == 0:
        print(f"✅ PASS: No rounding error (within {precision} decimal precision)")
    else:
        error = abs(lot_diff_sum - line.difference_qty)
        print(f"❌ FAIL: Rounding error detected!")
        print(f"   Difference: {error:.10f}")
        print(f"   This could cause posting to fail!")
    
    cr.rollback()

## CRITICAL TEST 5: All Negative Quants Scenario
### Risk: Line 223 - all_negative_quants = negative_quants and not positive_quants
### Line 230 - assigns full scanned qty to first negative lot

In [ ]:
with registry.cursor() as cr:
    env = api.Environment(cr, uid, context)
    
    print("=" * 70)
    print("CRITICAL TEST 5: All Negative Quants")
    print("=" * 70)
    
    product = env['product.product'].browse(CRITICAL_TEST_CONFIG['test_product_id'])
    location = env['stock.location'].browse(CRITICAL_TEST_CONFIG['location_id'])
    
    # Create only negative quants
    lots = []
    for i in range(1, 4):
        lot = env['stock.lot'].create({
            'name': f'NEG-LOT-{i}',
            'product_id': product.id,
            'company_id': CRITICAL_TEST_CONFIG['company_id'],
        })
        lots.append(lot)
    
    negative_quantities = [-10, -5, -3]
    base_date = datetime.now() - timedelta(days=5)
    
    print("Setup with ALL NEGATIVE quants:")
    total_negative = 0
    for i, (lot, qty) in enumerate(zip(lots, negative_quantities)):
        env['stock.quant'].create({
            'product_id': product.id,
            'location_id': location.id,
            'lot_id': lot.id,
            'quantity': qty,
            'in_date': base_date + timedelta(days=i),
            'company_id': CRITICAL_TEST_CONFIG['company_id'],
        })
        total_negative += qty
        print(f"  {lot.name}: {qty}")
    
    print(f"\nTotal (all negative): {total_negative}")
    
    # Scan 50 units
    adjustment = env['stock.adjustment.barcode'].create({
        'location_id': location.id,
        'company_id': CRITICAL_TEST_CONFIG['company_id'],
    })
    
    scan_qty = 50
    for i in range(scan_qty):
        adjustment.on_barcode_scanned(CRITICAL_TEST_CONFIG['test_product_barcode'])
    
    print(f"\nScanned: {scan_qty} units")
    
    # Confirm
    adjustment.action_confirm()
    
    line = adjustment.inv_adjustment_line_ids[0]
    
    print(f"\nAfter Confirm:")
    print(f"  On Hand: {line.on_hand_qty}")
    print(f"  Scanned: {line.total_scanned_qty}")
    print(f"  Difference: {line.difference_qty}")
    
    # Check lot lines
    print(f"\nLot Lines (CRITICAL - First lot should get full scanned qty):")
    first_lot_new_qty = None
    other_lots_zero = True
    
    for i, lot_line in enumerate(line.adjustment_line_lot_ids.sorted('lot_id')):
        print(f"  {lot_line.lot_id.name}:")
        print(f"    Current: {lot_line.current_qty}")
        print(f"    New: {lot_line.new_qty}")
        print(f"    Diff: {lot_line.difference_qty}")
        
        if i == 0:
            first_lot_new_qty = lot_line.new_qty
        else:
            if lot_line.new_qty != 0:
                other_lots_zero = False
    
    # CRITICAL VALIDATIONS
    print(f"\n{'='*70}")
    print("CRITICAL VALIDATIONS:")
    print(f"{'='*70}")
    
    precision = env['decimal.precision'].precision_get('Product Unit of Measure')
    
    # Per code line 230: first lot gets scanned qty when all negative
    if float_compare(first_lot_new_qty, scan_qty, precision_digits=precision) == 0:
        print(f"✅ PASS: First lot gets full scanned quantity ({scan_qty})")
    else:
        print(f"❌ FAIL: First lot new_qty={first_lot_new_qty}, expected {scan_qty}")
    
    # Other lots should be 0
    if other_lots_zero:
        print(f"✅ PASS: Other negative lots set to 0")
    else:
        print(f"❌ FAIL: Some non-first lots not set to 0")
    
    # Final stock should equal scanned
    final_stock = sum(line.adjustment_line_lot_ids.mapped('new_qty'))
    if float_compare(final_stock, scan_qty, precision_digits=precision) == 0:
        print(f"✅ PASS: Final stock equals scanned quantity")
    else:
        print(f"❌ FAIL: Final stock {final_stock} != Scanned {scan_qty}")
    
    # Lot totals match line difference
    lot_total = sum(line.adjustment_line_lot_ids.mapped('difference_qty'))
    if float_compare(lot_total, line.difference_qty, precision_digits=precision) == 0:
        print(f"✅ PASS: Lot totals match line difference")
    else:
        print(f"❌ FAIL: Lot total {lot_total} != Line diff {line.difference_qty}")
    
    cr.rollback()

## CRITICAL TEST 6: Backdated Refresh State Inconsistency
### Risk: Refresh in wrong state or after lots computed

In [ ]:
with registry.cursor() as cr:
    env = api.Environment(cr, uid, context)
    
    print("=" * 70)
    print("CRITICAL TEST 6: Backdated Refresh State Checks")
    print("=" * 70)
    
    product = env['product.product'].browse(CRITICAL_TEST_CONFIG['test_product_id'])
    location = env['stock.location'].browse(CRITICAL_TEST_CONFIG['location_id'])
    
    lot = env['stock.lot'].create({
        'name': 'STATE-LOT-001',
        'product_id': product.id,
        'company_id': CRITICAL_TEST_CONFIG['company_id'],
    })
    
    env['stock.quant'].create({
        'product_id': product.id,
        'location_id': location.id,
        'lot_id': lot.id,
        'quantity': 100,
        'in_date': datetime.now() - timedelta(days=10),
        'company_id': CRITICAL_TEST_CONFIG['company_id'],
    })
    
    # Test 1: Refresh in draft state
    print("\nTest 6.1: Refresh in DRAFT state")
    adjustment = env['stock.adjustment.barcode'].create({
        'location_id': location.id,
        'company_id': CRITICAL_TEST_CONFIG['company_id'],
    })
    
    for i in range(50):
        adjustment.on_barcode_scanned(CRITICAL_TEST_CONFIG['test_product_barcode'])
    
    adjustment.action_confirm()
    initial_diff = adjustment.inv_adjustment_line_ids[0].difference_qty
    
    print(f"  State: {adjustment.state}")
    print(f"  Initial difference: {initial_diff}")
    
    # Try refresh in to_approve state
    backdate = datetime.now() - timedelta(days=5)
    try:
        wizard = env['stock.refresh.stock.wizard'].with_context(
            active_model='stock.adjustment.barcode',
            active_id=adjustment.id
        ).create({
            'fetch_type': 'backdated',
            'backdate': backdate
        })
        wizard.action_apply()
        
        new_diff = adjustment.inv_adjustment_line_ids[0].difference_qty
        print(f"  ✅ Refresh allowed in {adjustment.state} state")
        print(f"  New difference: {new_diff}")
        
    except Exception as e:
        print(f"  ❌ Refresh failed: {str(e)}")
    
    # Test 2: Refresh in approved state
    print("\nTest 6.2: Refresh in APPROVED state")
    adjustment.action_approved()
    print(f"  State: {adjustment.state}")
    
    try:
        wizard = env['stock.refresh.stock.wizard'].with_context(
            active_model='stock.adjustment.barcode',
            active_id=adjustment.id
        ).create({
            'fetch_type': 'current'
        })
        wizard.action_apply()
        print(f"  ⚠️ WARNING: Refresh allowed in approved state (may cause issues)")
        
    except Exception as e:
        print(f"  ✅ Refresh blocked: {str(e)}")
    
    # Test 3: Check lot lines consistency after refresh
    print("\nTest 6.3: Lot lines consistency after multiple refreshes")
    
    line = adjustment.inv_adjustment_line_ids[0]
    lot_count_before = len(line.adjustment_line_lot_ids)
    
    # Apply refresh again
    try:
        wizard = env['stock.refresh.stock.wizard'].with_context(
            active_model='stock.adjustment.barcode',
            active_id=adjustment.id
        ).create({
            'fetch_type': 'backdated',
            'backdate': datetime.now() - timedelta(days=3)
        })
        wizard.action_apply()
        
        lot_count_after = len(line.adjustment_line_lot_ids)
        
        if lot_count_after == lot_count_before:
            print(f"  ✅ Lot line count consistent: {lot_count_after}")
        else:
            print(f"  ⚠️ Lot line count changed: {lot_count_before} → {lot_count_after}")
            print(f"     (May indicate lot lines not properly cleared/recreated)")
    except Exception as e:
        print(f"  Error: {str(e)}")
    
    cr.rollback()

## CRITICAL TEST 7: No Quants Found Scenario
### Risk: Lines 183-202 - fallback logic when no quants in location

In [ ]:
with registry.cursor() as cr:
    env = api.Environment(cr, uid, context)
    
    print("=" * 70)
    print("CRITICAL TEST 7: No Quants Found - Fallback Logic")
    print("=" * 70)
    
    # Create product without any quants
    new_product = env['product.product'].create({
        'name': 'No Quants Product',
        'type': 'product',
        'tracking': 'lot',
        'barcode': 'NO-QUANTS-001',
        'standard_price': 100.0,
    })
    
    location = env['stock.location'].browse(CRITICAL_TEST_CONFIG['location_id'])
    
    print(f"Product: {new_product.name} (tracking: {new_product.tracking})")
    print(f"No quants exist for this product\n")
    
    # Create adjustment and scan
    adjustment = env['stock.adjustment.barcode'].create({
        'location_id': location.id,
        'company_id': CRITICAL_TEST_CONFIG['company_id'],
    })
    
    # Scan the product
    for i in range(10):
        adjustment.on_barcode_scanned('NO-QUANTS-001')
    
    print(f"Scanned: 10 units")
    
    # Try to confirm - should trigger fallback logic
    print(f"\nConfirming (triggers quant gathering)...")
    
    try:
        adjustment.action_confirm()
        
        line = adjustment.inv_adjustment_line_ids.filtered(
            lambda l: l.product_id == new_product
        )
        
        if line:
            print(f"\nLine created:")
            print(f"  Product: {line.product_id.name}")
            print(f"  On Hand: {line.on_hand_qty}")
            print(f"  Scanned: {line.total_scanned_qty}")
            print(f"  Difference: {line.difference_qty}")
            print(f"  Has Error: {line.is_editable}")
            print(f"  Lot lines: {len(line.adjustment_line_lot_ids)}")
            
            if line.is_editable:
                print(f"\n✅ EXPECTED: Line marked as erroneous (no quants)")
            else:
                print(f"\n⚠️ UNEXPECTED: Line not marked as error despite no quants")
                
                # Check if fallback found lots elsewhere
                if line.adjustment_line_lot_ids:
                    print(f"   Fallback found lots:")
                    for lot_line in line.adjustment_line_lot_ids:
                        print(f"     - {lot_line.lot_id.name}")
        else:
            print(f"\n❌ FAIL: No line created for product")
        
    except ValidationError as e:
        if "no quants available" in str(e).lower():
            print(f"\n✅ EXPECTED: Raised validation error about no quants")
            print(f"   Error: {str(e)}")
        else:
            print(f"\n❌ UNEXPECTED ERROR: {str(e)}")
    except Exception as e:
        print(f"\n❌ UNEXPECTED ERROR: {str(e)}")
    
    cr.rollback()

## Summary and Recommendations

### Critical Issues Found:

1. **Backdated Refresh Logic**: Stock movements between backdate and present may cause mismatch
2. **FIFO Index Bounds**: Potential IndexError in while loop
3. **Mixed Quants Handling**: Complex logic with edge cases
4. **Precision Rounding**: Accumulation of rounding errors
5. **All Negative Quants**: Special case handling
6. **State Management**: Refresh allowed in wrong states
7. **No Quants Fallback**: Error handling inconsistent

### Recommendations:

1. **Add bounds checking** in FIFO deduction loop (line 265-274)
2. **Use Decimal** instead of float for financial calculations
3. **Add state validation** in refresh wizard
4. **Improve error messages** for lot selection failures
5. **Add transaction logging** for debugging backdated issues
6. **Validate lot totals** before posting
7. **Add warning** when backdating beyond stock movement dates